# Eval
### Imports

In [1]:
#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
import torch.optim as optim
# Utils
import numpy as np
from numpy import ndarray
import logging
# Base Scripts
from Libraries.VAE import *
from Libraries.Utils import *
from Conf import *

### Config

In [2]:
model_name: str = "conv_VAE_v1"
training_data_name: str = "training_v2"
logging_level: int = LIGHT_DEBUG
logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

### Functions

In [3]:
def generate_data(model: nn.Module, device: str,  n_samples: int = 1, seed: ndarray = None, file_name: str = "test") -> None:
    model.eval()
    if seed is not None:
        save_audio_file(spectrogram_to_audio(unnormalize(seed), LEN_FFT), f"{RESULT_PATH}/{file_name}_seed.wav")
        samples = generate_sample(model, device, Tensor(seed).view(1, 1, seed.shape[-2], seed.shape[-1]), num_samples=n_samples)
    else:
        samples = generate_sample(model, device, num_samples=n_samples)
    for i in range(samples.shape[1]):
        save_audio_file(spectrogram_to_audio(unnormalize(samples[0,i]), LEN_FFT), f"{RESULT_PATH}/{file_name}_{i:02d}.wav", SAMPLE_RATE)
    logger.light_debug(f"Saved {samples.shape[0]} samples to {RESULT_PATH}")


def pass_through(model: nn.Module, device: str, sample: ndarray, file_name: str = "test") -> None:
    save_audio_file(spectrogram_to_audio(unnormalize(sample), LEN_FFT), f"{RESULT_PATH}/{file_name}_inp.wav")
    x = fwd_pass(model, device, Tensor(sample).view(1, 1, sample.shape[-2], sample.shape[-1]))
    save_audio_file(spectrogram_to_audio(unnormalize(x[0,0]), LEN_FFT), f"{RESULT_PATH}/{file_name}_out.wav")
    logger.light_debug(f"Saved passed through sample to {RESULT_PATH}")

### Data Loading & Model Creation

In [7]:
file = load_training_data(f"{DATA_PATH}/{training_data_name}.npy")
logger.info(f"Data loaded with shape: {file.shape}")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VAE(in_channels=1, latent_dim=256, device=device,input_shape=[0,0, file.shape[-2], file.shape[-1]], n_conv_blocks=1, n_starting_filters=32, lin_bottleneck=False).to(device)
model.load_state_dict(torch.load(f"{MODEL_PATH}/{model_name}.pth", map_location=device))

2025-02-22 00:39:00,411 - LIGHT_DEBUG - Ndarray loaded from ../Data/training_v2.npy of shape: (4251, 1024, 672)
2025-02-22 00:39:00,991 - INFO - Data loaded with shape: (4251, 1024, 672)
C:\Users\finia\AppData\Local\Temp\ipykernel_23416\1614215680.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don

<All keys matched successfully>

### Evaluation

In [9]:
#generate_data(model=model, seed=file[100])
pass_through(model, device, file[10])

2025-02-22 00:39:33,938 - LIGHT_DEBUG - Unnormalized to range: [-50,50]
2025-02-22 00:39:33,941 - LIGHT_DEBUG - Started GL
2025-02-22 00:39:43,131 - LIGHT_DEBUG - Reconstructed audio: (343552,)
2025-02-22 00:39:43,139 - LIGHT_DEBUG - Saved file to:../Results/test_inp.wav
2025-02-22 00:39:43,189 - LIGHT_DEBUG - Started passthrough
2025-02-22 00:39:43,823 - LIGHT_DEBUG - Created samples: (1, 1, 1024, 672)
2025-02-22 00:39:43,828 - LIGHT_DEBUG - Unnormalized to range: [-50,50]
2025-02-22 00:39:43,829 - LIGHT_DEBUG - Started GL
2025-02-22 00:39:46,186 - LIGHT_DEBUG - Reconstructed audio: (343552,)
2025-02-22 00:39:46,192 - LIGHT_DEBUG - Saved file to:../Results/test_out.wav
2025-02-22 00:39:46,194 - LIGHT_DEBUG - Saved passed through sample to ../Results
